# Generate neutral and selected SFS vectors of all transcripts (=gene)

#### Method:

1. Get all unique PAC id's from ggf3 file `/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/Creinhardtii_v5.3_223_gene.gff3.gz`
    - Each PAC id is associated with a unique transcript.
2. Get chromosome, start, and end positions of each `five_prime_UTR`, `CDS`, and `three_prime_UTR` in each PAC id. Store as list `exon_positions` and export to working folder.
    - `exon_positions`: `<list>` 
    
       `[[(chromosome <str>, start <int>, end <str>),...,(chromosome <str>, start <int>, end <str>)], ...,`
       
       `[(chromosome <str>, start <int>, end <str>),...,(chromosome <str>, start <int>, end <str>)]]`
3. Pass `exon_positions` through `SFSs_from_annotation()` to get SFS vectors for each transcript for neutral and selected sites.
4. Pass the dictionary output of SFS vectors from `SFSs_from_annotation()` through `reduce_SFS()` to reduce SFS vectors into one SFS vector. Store all reduced vector outputs as lists of vectors -- `selected_SFS` and `neutral_SFS`.
5. Create dataframe of reduced SFS for each transcript.

| PAC_id  	| selected_SFS        	| neutral_SFS         	|
|---------	|---------------------	|---------------------	|
| `<str>` 	| `[<int>,...,<int>]` 	| `[<int>,...,<int>]` 	|


In [1]:
import sys
sys.path.append('../../scripts/')
#sys.path
import pandas as pd
import Search_algorithms as sag
import gzip
import math
import re
import time
import numpy as np
import pickle
from functools import partial
import psutil #psutil-5.7.0
from multiprocessing import Pool #multiprocessing-2.6.2.1

### 1. Get all unique PAC id's from ggf3 file

In [2]:
gff_file_path='/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/Creinhardtii_v5.3_223_gene.gff3.gz'
merged = pd.read_csv("../../data/intermediate_data_02/sampled_genes.csv") 

In [3]:
merged[:5]

,num_detected,num_manipulated,num_sampled,source,transcript_id,annotation_version,gene_id,gene_symbol,pathway_id,transcript_id_v5.3.1,PAC_id
0,0.0,0,1.0,Bajhaiya_2016,Cre01.g000017.t1.1,v5.5,Cre01.g000017,NaN,NaN,g2.t1,PAC:26903746
1,0.0,0,1.0,Kwak_2017,Cre01.g000017.t1.1,v5.5,Cre01.g000017,NaN,NaN,g2.t1,PAC:26903746
2,1.0,0,1.0,Bajhaiya_2016,Cre01.g000033.t1.1,v5.5,Cre01.g000033,NaN,NaN,g3.t1,PAC:26903463
3,0.0,0,1.0,Gargouri_2015,Cre01.g000050.t1.1,v5.5,Cre01.g000050,NaN,NaN,Cre01.g000050.t1.3,PAC:26903339
4,0.0,0,1.0,Bajhaiya_2016,Cre01.g000100.t1.1,v5.5,Cre01.g000100,NaN,NaN,Cre01.g000100.t1.3,PAC:26903974


In [4]:
def tryextract(i, pattern):
    
    '''This function takes in a string and returns the first matching group in the search pattern'''
    
    try: 
        m = re.search(pattern, i).group(1)
        return(m)
    
    except AttributeError:
        return(None)

In [5]:
#Import gff as working dataframe. Extract Name and PAC id from the column 'attribute' as separate columns
with gzip.open(gff_file_path, "rt", encoding="utf-8") as z:
    
    df = pd.read_csv(z,delimiter=r"\s+",skiprows=1,header=None)
    df.columns = ['chromosome', 'source', 'feature', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    df['Name'] = df.attributes.apply(lambda x: tryextract(x, r"Name=(.+);pacid"))
    df['ID'] = df.attributes.apply(lambda x: tryextract(x, r"ID=(PAC:[0-9]+)"))
    

In [6]:
pac_id = list(np.unique(df.ID.dropna()))

In [7]:
df[:5]

,chromosome,source,feature,start,end,score,strand,phase,attributes,Name,ID
0,chromosome_1,phytozome8_0,gene,24026,30617,.,+,.,ID=Cre01.g000050;Name=Cre01.g000050,None,None
1,chromosome_1,phytozome8_0,mRNA,24026,30617,.,+,.,ID=PAC:26903339;Name=Cre01.g000050.t1.3;pacid=...,Cre01.g000050.t1.3,PAC:26903339
2,chromosome_1,phytozome8_0,five_prime_UTR,24026,24125,.,+,.,ID=PAC:26903339.five_prime_UTR.1;Parent=PAC:26...,None,PAC:26903339
3,chromosome_1,phytozome8_0,CDS,24126,28105,.,+,0,ID=PAC:26903339.CDS.1;Parent=PAC:26903339;paci...,None,PAC:26903339
4,chromosome_1,phytozome8_0,CDS,28291,28644,.,+,1,ID=PAC:26903339.CDS.2;Parent=PAC:26903339;paci...,None,PAC:26903339


### 2. Get chromosome, start, and end positions of each five_prime_UTR, CDS, and three_prime_UTR in each PAC id. 

In [8]:
t0 = time.time()
exon_positions = []
features_condition = ['five_prime_UTR', 'CDS', 'three_prime_UTR']
#keep gff rows that contain CDS positions, remove gff rows that are missing PAC_ID's
filtered_df = df[df.feature.isin(features_condition)].dropna(subset = ['ID'])
#loop through genes in gff with PAC_ID's; transcript filtered by condition variable
count = 0
for pac_index in range(0,len(pac_id)): 
    
    temp = filtered_df[filtered_df.ID==pac_id[pac_index]]
    #print(transcript)
    if len(temp)== 0: 
        exon_positions.append(None)
        count+=0
        
    else:
        exon_positions.append([])
        #add CDS coordinates as tuple (chromosome, start, end) to gene_set
        for i in list(temp.index):
            exon_positions[pac_index].append((temp.chromosome[i], temp.start[i], temp.end[i]))
        
t1 = time.time()
print("Time taken", t1-t0, "s")

Time taken 197.6272006034851 s


In [9]:
print("Number of PACid with missing  exon positions: " + str(count) )

Number of PACid with missing  exon positions: 0


##### Export `exon_positions` to working folder.

In [10]:
with open('../../data/intermediate_data_02/exon_positions.pk', 'wb') as f:
    pickle.dump(exon_positions, f)

### 2b. Add PAC ID to `sampled_genes.csv`

In [11]:
subset = df[df.feature =="mRNA"].sort_values(by="Name").reset_index()
search_list = list(subset.Name)

In [12]:
#Order list of matching PAC id's to 'transcript_id_v5.3.1'
#import re
t0 = time.time()
pacid_list = []

for row in range(0,len(merged)):
    transcript_id = merged.loc[row,'transcript_id_v5.3.1']
    pacid = None
    
    index = sag.BinarySearch(search_list, transcript_id)
    
    if index > -1: 
        pacid = subset.loc[index, "ID"]
                    
    pacid_list.append(pacid)

t1 = time.time()
print("Time taken", t1-t0, "s")

Time taken 12.20185136795044 s


In [13]:
merged['PAC_id'] = pacid_list

In [14]:
merged[:5]

,num_detected,num_manipulated,num_sampled,source,transcript_id,annotation_version,gene_id,gene_symbol,pathway_id,transcript_id_v5.3.1,PAC_id
0,0.0,0,1.0,Bajhaiya_2016,Cre01.g000017.t1.1,v5.5,Cre01.g000017,NaN,NaN,g2.t1,PAC:26903746
1,0.0,0,1.0,Kwak_2017,Cre01.g000017.t1.1,v5.5,Cre01.g000017,NaN,NaN,g2.t1,PAC:26903746
2,1.0,0,1.0,Bajhaiya_2016,Cre01.g000033.t1.1,v5.5,Cre01.g000033,NaN,NaN,g3.t1,PAC:26903463
3,0.0,0,1.0,Gargouri_2015,Cre01.g000050.t1.1,v5.5,Cre01.g000050,NaN,NaN,Cre01.g000050.t1.3,PAC:26903339
4,0.0,0,1.0,Bajhaiya_2016,Cre01.g000100.t1.1,v5.5,Cre01.g000100,NaN,NaN,Cre01.g000100.t1.3,PAC:26903974


In [15]:
with open('../../data/workflow/sampled_genes.pk', 'wb') as f:
    pickle.dump(merged, f)

### 3. Define `SFSs_from_annotation()` to get SFS vectors for each transcript for neutral and selected sites.

In [16]:
exon_positions = pickle.load(open("../../data/intermediate_data_02/exon_positions.pk", "rb"))

In [17]:
#This code is from Rob
from ness_vcf import SFS
from pysam import TabixFile
from annotation import annotation_table
#our SFS in neutral  is a fold4 site
#fold0 sites go in selected SFS vector in est_dfe

def SFSs_from_annotation(coordinates, min_alleles=None, fold0_only=False, fold4_only=False): 
    annotation_tabix = TabixFile(filename="/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/concatenated_GFF/annotation_table.txt.gz")
    """
    This function will return a dictionary of SFS objects
    The dictionary will contain one SFS for each number of alleles that can be called
        ie min_alleles to total number of individuals sequenced
    It is possible to combine these SFSs by:
        rounding MAF * (number of individuals sequenced) and keeping only one SFS
    Arguments:
     - take a TabixFile of the annotation table
     - the chromosome, start and end (1-based inclusive)
     - an optional minimum number of alleles - below this the site is shiite so don't take a bite
     - neutral_only skips sites that aren't intergenic, intronic or 4-fold degenerate
    """
    SFSs = {}
    
    #Loop through input list of (chromosome, start, end) from gff3
    for i in coordinates:
        chromosome, start, end = i
        for line in annotation_tabix.fetch(chromosome, start-1, end):
        # `annotation_line` is a class that has all the annotation table columns as attributes
            #print(line.split())
            a = annotation_table.annotation_line(line) #a has a lot of attributes
            allele_counts = a.quebec_alleles
            if fold0_only and int(a.fold0) == 0: #skip sites that are not fold0
                continue
            if fold4_only and int(a.fold4) == 0: #skip sites that are not fold4
                continue
            try:
                MAF, total_alleles_called  = MAF_from_allele_count(allele_counts,min_alleles=min_alleles)
                #if MAF > 0: print(MAF, total_alleles_called)
            except TypeError: 
                continue
            if min_alleles != None and total_alleles_called < min_alleles: #filter sites with too few alleles called
                continue
            if total_alleles_called not in SFSs: 
                #make SFS dictionary where key = n for SFS vector of length n
                #you can't feed program with different alleles, so try to standardize and round them somehow in proportion to max alleles
                SFSs[total_alleles_called] = SFS([0]*(total_alleles_called+1))
            SFSs[total_alleles_called].add(MAF,total_alleles_called)


    return SFSs


def MAF_from_allele_count(allele_counts, min_alleles=None): # num of rare alleles/num of total alleles
    """
    return the minor allele frequency and the number of called alleles
    take a single allele_counts from annotation table ie, A:C:G:T    
    optionally min_alleles will filter sites with too few alleles called
    """
    minor_allele_count = sorted([int(i) for i in allele_counts.split(":")])[-2]
    total_alleles_called = sum([int(i) for i in allele_counts.split(":")])
    if min_alleles != None and total_alleles_called <= min_alleles:
        return None
    try:
        MAF = minor_allele_count/float(total_alleles_called)
        return (MAF,total_alleles_called)
    except ZeroDivisionError:
        return None
    

##annotation_table has clones filtered out

### Do SFS vectors from `SFS_annotation()` include invariant sites?
SFS vectors from SFS dictionary output of `SFSs_from_annotation()` contains the number of invariants as the first element.

In [18]:
selected = SFSs_from_annotation(exon_positions[1],min_alleles=12, fold0_only=True, fold4_only=False)
neutral = SFSs_from_annotation(exon_positions[1],min_alleles=12, fold0_only=False, fold4_only=True)

In [19]:
print ("Alleles", "\t", "SFS vector")
for key in selected.keys():
    print (selected[key].alleles, "\t\t", selected[key].sfs)

Alleles 	 SFS vector
16 		 [19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17 		 [5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18 		 [375, 8, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
print ("Alleles", "\t","Invariant sites", "\t", "SFS vector")
for key in neutral.keys():
    print (neutral[key].alleles, "\t\t", neutral[key].invariant(), "\t\t\t", neutral[key].sfs)

Alleles 	 Invariant sites 	 SFS vector
16 		 4 			 [4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17 		 2 			 [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18 		 120 			 [120, 8, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 4. Define `reduce_SFS()` to reduce SFS vectors into one SFS vector

In [21]:
def reduce_SFS(SFS_dict, max_alleles = 18):
    
    '''This function aggregates SFS vectors from SFSs_from_annotation() dictionary output into one SFS vector.
    Default max_alleles = 18'''
    
    ref_SFS = range(0,max_alleles)
    new_sfs = [0]*max_alleles
    for key in SFS_dict.keys():
        
        sfs_length = len(SFS_dict[key].sfs)
        #print(key)
        normalized_index = [round(i/sfs_length*max_alleles)-1 for i in range(1,sfs_length+1)]
        #print(normalized_index)
        for i in range(0,sfs_length): new_sfs[normalized_index[i]]+=SFS_dict[key].sfs[i]
        
    return(new_sfs)

##### Test `reduce_SFS()`

In [22]:
#Test function logic
max_alleles =10
SFS_dict = {10: [1,0,1,0,1,0,1,0,1,0],
           5: [1,0,1,0,1]}
expected_output = [1,1,1,0,1,1,1,0,1,1]
ref_SFS = range(0,max_alleles)
new_sfs = [0]*max_alleles

for key in SFS_dict.keys():
        
        sfs_length = len(SFS_dict[key])
        #print(key)
        normalized_index = [round(i/sfs_length*max_alleles)-1 for i in range(1,sfs_length+1)]
        #print(normalized_index)
        for i in range(0,sfs_length): new_sfs[normalized_index[i]]+=SFS_dict[key][i]

if new_sfs==expected_output: print("reduce_SFS() logic worked as expected")
else: print("reduce_SFS() logic did not work as expected")


reduce_SFS() logic worked as expected


In [23]:
#Test if collapse_SFS() returns error
reduce_SFS(neutral)

[126, 9, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### Estimate time required to run `SFSs_from_annotation()` and `reduce_SFS()` on all transcripts. 

#### 1. One process

In [24]:
t0 = time.time()
selected_SFS = []
neutral_SFS = []
#for i in range(len(exon_positions)):
for i in range(0,100):
    coordinates = exon_positions[i]
    if coordinates != None:
        selected = SFSs_from_annotation(coordinates, min_alleles=12, fold0_only=True, fold4_only=False)
        selected_SFS.append(reduce_SFS(selected, max_alleles = 18))
        neutral = SFSs_from_annotation(coordinates, min_alleles=12, fold0_only=False, fold4_only=True)
        neutral_SFS.append(reduce_SFS(neutral, max_alleles = 18))
t1 = time.time()
print("100 genes took:", t1-t0, "seconds.")
print("Estimated calculation time:", len(exon_positions)/100*(t1-t0)/60, "minutes")

100 genes took: 16.483078718185425 seconds.
Estimated calculation time: 53.64967404790719 minutes


In [25]:
print(selected_SFS[:10])
print(neutral_SFS[:10])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [399, 9, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [921, 22, 15, 9, 3, 6, 1, 1, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2623, 26, 7, 5, 0, 4, 2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], [478, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [892, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2015, 38, 10, 10, 3, 6, 2, 6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [2000, 38, 10, 10, 3, 6, 2, 6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [851, 4, 7, 4, 2, 1, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [126, 9, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], [273, 12, 11, 3, 5, 6, 6, 3, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0], [791, 38, 16, 8, 8, 8, 8, 4, 10, 1, 0, 0, 0, 0, 0, 0, 0, 0], [127, 4, 4, 1, 1, 6, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [228, 5, 3, 2, 3, 2, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], [530, 31, 14, 16, 6, 13, 9, 5, 2, 1, 0, 0, 0, 0, 0, 0, 0, 

#### 2. One process using multiprocessing.

In [26]:
t0 = time.time()
def main():
    pool = Pool(processes=1)
    
    exon_portion = exon_positions[0:100]
    
    func = partial(SFSs_from_annotation, min_alleles=12, fold0_only=True, fold4_only=False)
    selected = list(pool.map(func, exon_portion))
    
    func = partial(SFSs_from_annotation, min_alleles=12, fold0_only=False, fold4_only=True)
    neutral = list(pool.map(func, exon_portion))
    
    func = partial(reduce_SFS, max_alleles = 18)
    reduced_selected = list(pool.map(func, selected))
    
    func = partial(reduce_SFS, max_alleles = 18)
    reduced_neutral = list(pool.map(func, neutral))
    
    return reduced_selected, reduced_neutral

if __name__ == '__main__':
    reduced_selected, reduced_neutral = main()
    
t1 = time.time()
print("100 genes took:", t1-t0, "seconds.")
print("Estimated calculation time:", len(exon_positions)/100*(t1-t0)/60, "minutes")

100 genes took: 16.567466020584106 seconds.
Estimated calculation time: 53.9243406526645 minutes


#### 3. Eight processes using multiprocessing.

In [27]:
print("Number of CPUs: ", psutil.cpu_count(logical=False))

Number of CPUs:  12


In [28]:
t0 = time.time()
def main():
    pool = Pool(processes=8)
    
    exon_portion = exon_positions[0:100]
    
    func = partial(SFSs_from_annotation, min_alleles=12, fold0_only=True, fold4_only=False)
    selected = list(pool.map(func, exon_portion))
    
    func = partial(SFSs_from_annotation, min_alleles=12, fold0_only=False, fold4_only=True)
    neutral = list(pool.map(func, exon_portion))
    
    func = partial(reduce_SFS, max_alleles = 18)
    reduced_selected = list(pool.map(func, selected))
    
    func = partial(reduce_SFS, max_alleles = 18)
    reduced_neutral = list(pool.map(func, neutral))
    
    return reduced_selected, reduced_neutral

if __name__ == '__main__':
    reduced_selected, reduced_neutral = main()
    
t1 = time.time()
print("100 genes took:", t1-t0, "seconds.")
print("Estimated calculation time:", len(exon_positions)/100*(t1-t0)/60, "minutes")

100 genes took: 2.58024001121521 seconds.
Estimated calculation time: 8.398251196503638 minutes


### Run the multiprocessing script on all transcripts in `exon_positions`.

In [29]:
t0 = time.time()
def main():
    pool = Pool(processes=8)
    
    exon_portion = exon_positions
    
    func = partial(SFSs_from_annotation, min_alleles=12, fold0_only=True, fold4_only=False)
    selected = list(pool.map(func, exon_portion))
    
    func = partial(SFSs_from_annotation, min_alleles=12, fold0_only=False, fold4_only=True)
    neutral = list(pool.map(func, exon_portion))
    
    func = partial(reduce_SFS, max_alleles = 18)
    reduced_selected = list(pool.map(func, selected))
    
    func = partial(reduce_SFS, max_alleles = 18)
    reduced_neutral = list(pool.map(func, neutral))
    
    return reduced_selected, reduced_neutral

if __name__ == '__main__':
    reduced_selected, reduced_neutral = main()
    
t1 = time.time()
print("Script took:", t1-t0, "seconds.")

Script took: 490.44499588012695 seconds.


## 5. Create dataframe of reduced SFS for each transcript.

Export `reduced_SFS` dataframe as `../../data/intermediate_data_02/reduced_SFS.pk`

In [30]:
reduced_SFS = pd.DataFrame({'PAC_id': pac_id,
              'selected_SFS': reduced_selected,
              'neutral_SFS': reduced_neutral})

with open('../../data/intermediate_data_02/reduced_SFS.pk', 'wb') as f:
    pickle.dump(reduced_SFS, f)